NGVS paper: https://iopscience.iop.org/article/10.3847/1538-4357/ab339f#apjab339ft4

Virgo (1 R_vir) covers 104 deg2 sky. This paper only searched 3.71 deg2 (0.3 Mpc2) around M87. They use CFHT/MegaCam.

In [2]:
# Import packages
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, hstack, vstack

In [5]:
import kuaizi as kz

In [6]:
env_dict = {'project': 'HSC', 'name': 'LSBG', 'data_dir': '/scratch/gpfs/jiaxuanl/Data'}
kz.utils.set_env(**env_dict)
kz.set_matplotlib(style='JL', usetex=True, dpi=100, fontsize=16)

Table 4: Basic Parameters

In [144]:
cat1 = Table.read('./Catalog/NGVS/XIV/table4_data.txt', 
                  format='ascii.fixed_width_no_header',
                 col_starts=(0, 28, 40, 51, 54, 57, 63, 64, 67, 70, 75, 77, 82, 89, 95, 100, 106, 111),
                 )
new_cols = ['name', 'ra', 'dec', 'RAh', 'RAm', 'RAs', 
            'DE-', 'DEd', 'DEm', 'DEs', 'Class', 'VCC', 
            'Vel', 'e_Vel', 'r_Vel', 'E(B-V)', 'FWHM', 'Det']

cat1.rename_columns(cat1.colnames, new_cols)

In [145]:
cat2 = Table.read('./Catalog/NGVS/XIV/table5_data.txt', 
                  format='ascii.fixed_width_no_header', 
                  col_starts=(0, 28, 30, 36, 41, 47, 53, 59, 65, 71, 75, 82, 88, 94, 95, 97),
                  )
new_cols = ['name', 'Class', 'gmagCG', 'C80/20', 'reCG', 'mue', 
            '<mue>', 'mu0', 'gmagS', 'n', 'reS', 
            'mueS', '<mueS>', 'NucID', 'u_NucID', 'gmagNuc']
cat2.rename_columns(cat2.colnames, new_cols)

In [146]:
cat3 = Table.read('./Catalog/NGVS/XIV/table6_data.txt', 
                  format='ascii.fixed_width_no_header', 
                  col_starts=(0, 29, 31, 36, 41, 47, 53, 59, 65, 70, 75, 81),
                 )
new_cols = ['name', 'Class', '<eps>T', 'e_<eps>T', 'thetaT', 'e_thetaT', 
            '<B4>T', 'e_<B4>T', '<eps>GF', 'e_<eps>GF', 'thetaGF', 
            'e_thetaGF']
cat3.rename_columns(cat3.colnames, new_cols)

In [147]:
cat4 = Table.read('./Catalog/NGVS/XIV/table7_data.txt', 
                  format='ascii.fixed_width_no_header', 
                  col_starts=(0, 28, 30, 36, 41, 47, 53),
                 )
new_cols = ['name', 'Class', 'gmag', 'u-g', 'g-r', 'g-i', 
            'g-z']
cat4.rename_columns(cat4.colnames, new_cols)

In [148]:
cat1.sort('name')
cat2.sort('name')
cat3.sort('name')
cat4.sort('name')

In [149]:
cat2.remove_columns(['name', 'Class'])
cat3.remove_columns(['name', 'Class'])
cat4.remove_columns(['name', 'Class'])

In [150]:
cat = hstack([cat1, cat2, cat3, cat4])

In [153]:
def post_process_cat_NGVS(input_cuts_cat):
    """
    Calculate physical properties assuming a distance of 16.5 Mpc
    """
    import joblib
    import astropy.units as u
    import pickle

    cuts_cat = input_cuts_cat.copy()
    
    # Write to catalog
    cuts_cat['host_ang_diam_dist'] = 16.5 * u.Mpc
    # Physical sizes
    # not consider peculiar motion
    ang_diam_dist = cuts_cat['host_ang_diam_dist'].data
    R_e_phys = cuts_cat['reS'].data / \
        206265 * ang_diam_dist * 1000  # in kpc
    cuts_cat['rhalf_phys'] = R_e_phys * u.kpc

    # Absolute magnitudes
    cuts_cat['abs_mag_g'] = cuts_cat['gmagS'] - 25 - 5 * \
        np.log10(ang_diam_dist)  # griz

    # average over g-i and g-r results
    log_ML_g = np.array([1.297 * cuts_cat['g-i'].data - 0.855,
                         1.774 * cuts_cat['g-r'].data - 0.783]).mean(axis=0)
    cuts_cat['log_ML_g'] = log_ML_g

    cuts_cat['log_m_star'] = -0.4 * \
        (cuts_cat['abs_mag_g'].data - 5.03) + log_ML_g
    
    return cuts_cat

In [154]:
cat = post_process_cat_NGVS(cat)

In [157]:
cat.write('./Catalog/NGVS/XIV/NGVS_core_satellite_cat.fits', format='fits', overwrite=True)